In [1]:
%matplotlib 

import numpy as np

from collections import defaultdict
from datetime import datetime
from matplotlib import pyplot as plt

with open('./test-3600.log') as f:
    lines = f.readlines()
    
logs = []
for l in lines:
    day, _, rest = l.partition(' ')
    hour, _, rest = rest.partition(' ')
    datestr = day + ' ' + hour
    date = datetime.strptime(datestr, "%Y-%m-%d %H:%M:%S,%f").timestamp()
    level, _, msg = rest.partition(' ')
    logs.append((date, level, msg.strip()))
    
len(logs)

Using matplotlib backend: MacOSX


2119084

In [2]:
recv_log, send_log, dxl_error_log, other_log = [], [], [], []

for t, lvl, msg in logs:
    if msg.startswith('Got msg '):
        recv_log.append((t, lvl, eval(msg[8:])))
    elif msg.startswith('Sending '):
        send_log.append((t, lvl, eval(msg[8:])[3:]))
    elif msg.startswith('Dynamixel error'):
        dxl_error_log.append((t, lvl, msg))
    else:
        other_log.append((t, lvl, msg))

len(recv_log), len(send_log), len(dxl_error_log), len(other_log)

(1057415, 1058097, 3570, 2)

In [3]:
set([msg for _, _, msg in dxl_error_log])

{'Dynamixel error 256 on motor id=30!'}

In [4]:
dxl_recv, orbita_recv = [], []
for t, lvl, msg in recv_log:
    if msg[0] == 15:
        dxl_recv.append((t, lvl, msg))
    elif msg[0] == 55:
        orbita_recv.append((t, lvl, msg))
    else:
        print('Unknown', msg)
        
len(dxl_recv), len(orbita_recv)

(707422, 349993)

In [5]:
from reachy_pyluos_hal.orbita import OrbitaRegister

orbita_pos_update = defaultdict(list)
dxl_pos_update = defaultdict(list)

for t, _, msg in orbita_recv:
    orbita_id = msg[1]
    orbita_reg = msg[2]
    
    if orbita_reg == OrbitaRegister.present_position.value:
        orbita_pos_update[orbita_id].append(t)
        
for t, _, msg in dxl_recv:
    if msg[1] == 36 or msg[1] == 37:
        dxl_id = msg[3]
        dxl_pos_update[dxl_id].append(t)

plt.figure()
for id, t in dxl_pos_update.items():
    plt.hist(np.diff(t), label=f'dxl {id}')
for id, t in orbita_pos_update.items():
    plt.hist(np.diff(t), label=f'orbita {id}')
plt.legend()

In [21]:
send_orbita_get, send_orbita_set = [], []
send_dxl_get, send_dxl_set = [], []

for t, lvl, msg in send_log:
    if msg[0] == 50:
        send_orbita_get.append((t, lvl, msg))
    elif msg[0] == 51:
        send_orbita_set.append((t, lvl, msg))
    elif msg[0] == 10:
        send_dxl_get.append((t, lvl, msg))        
    elif msg[0] == 11:
        send_dxl_set.append((t, lvl, msg))
    elif msg[0] == 200:
        continue
    else:
        print('Unknown', msg)
        
print('dxl get', len(send_dxl_get), 'set', len(send_dxl_set))
print('orbita get', len(send_orbita_get), 'set', len(send_orbita_set))

dxl get 2 set 527253
orbita get 2 set 527252


In [27]:
send_dxl_set[10]

(1612533148.865, 'DEBUG', [11, 30, 2, 30, 74, 2])

In [30]:
from reachy_pyluos_hal.orbita import OrbitaRegister

orbita_pos_set = defaultdict(list)
dxl_pos_set = defaultdict(list)

for t, _, msg in send_orbita_set:
    orbita_id = msg[1]
    orbita_reg = msg[2]
    
    if orbita_reg == OrbitaRegister.goal_position.value:
        orbita_pos_set[orbita_id].append(t)
        
for t, _, msg in send_dxl_set:
    if msg[1] == 30:
        dxl_id = msg[3]
        dxl_pos_set[dxl_id].append(t)

plt.figure()
for id, t in dxl_pos_set.items():
    plt.plot(np.diff(t), label=f'dxl {id}')
for id, t in orbita_pos_set.items():
    plt.plot(np.diff(t), label=f'orbita {id}')
plt.legend()

In [29]:
orbita_pos_set[10][:20]

[1612533148.817,
 1612533148.818,
 1612533148.824,
 1612533148.829,
 1612533148.836,
 1612533148.843,
 1612533148.851,
 1612533148.858,
 1612533148.865,
 1612533148.872,
 1612533148.879,
 1612533148.886,
 1612533148.894,
 1612533148.899,
 1612533148.906,
 1612533148.914,
 1612533148.921,
 1612533148.929,
 1612533148.935,
 1612533148.943]